In [22]:
import pandas as pd
import numpy as np

In [23]:
amazon_prime = pd.read_csv('C:/Users/Predator/Desktop/Lauti/Programación/GitHub/1. Data Engineering/#1/Datasets/amazon_prime_titles.csv', sep=',')
disney_plus = pd.read_csv('C:/Users/Predator/Desktop/Lauti/Programación/GitHub/1. Data Engineering/#1/Datasets/disney_plus_titles.csv', sep=',')
hulu = pd.read_csv('C:/Users/Predator/Desktop/Lauti/Programación/GitHub/1. Data Engineering/#1/Datasets/hulu_titles.csv', sep=',')
netflix = pd.read_json('C:/Users/Predator/Desktop/Lauti/Programación/GitHub/1. Data Engineering/#1/Datasets/netflix_titles.json')

In [24]:
#Column asignation
amazon_prime['Plataforma'] = 'Amazon_Prime'
disney_plus['Plataforma'] = 'Disney_Plus'
hulu['Plataforma'] = 'Hulu'
netflix['Plataforma'] = 'Netflix'

In [25]:
df_definitive = pd.concat([amazon_prime, disney_plus, hulu, netflix], axis=0)

In [26]:
df_definitive['director'].fillna('no data', inplace=True)
df_definitive['cast'].fillna('no data', inplace=True)
df_definitive['country'].fillna('no data', inplace=True)
df_definitive['date_added'].fillna('no data', inplace=True)
df_definitive['rating'].fillna('no data', inplace=True)
df_definitive['description'].fillna('no data', inplace=True)

In [27]:
df_definitive['rating'].unique()

array(['no data', '13+', 'ALL', '18+', 'R', 'TV-Y', 'TV-Y7', 'NR', '16+',
       'TV-PG', '7+', 'TV-14', 'TV-NR', 'TV-G', 'PG-13', 'TV-MA', 'G',
       'PG', 'NC-17', 'UNRATED', '16', 'AGES_16_', 'AGES_18_', 'ALL_AGES',
       'NOT_RATE', 'TV-Y7-FV', 'NOT RATED', '2 Seasons', '93 min',
       '4 Seasons', '136 min', '91 min', '85 min', '98 min', '89 min',
       '94 min', '86 min', '3 Seasons', '121 min', '88 min', '101 min',
       '1 Season', '83 min', '100 min', '95 min', '92 min', '96 min',
       '109 min', '99 min', '75 min', '87 min', '67 min', '104 min',
       '107 min', '84 min', '103 min', '105 min', '119 min', '114 min',
       '82 min', '90 min', '130 min', '110 min', '80 min', '6 Seasons',
       '97 min', '111 min', '81 min', '49 min', '45 min', '41 min',
       '73 min', '40 min', '36 min', '39 min', '34 min', '47 min',
       '65 min', '37 min', '78 min', '102 min', '129 min', '115 min',
       '112 min', '61 min', '106 min', '76 min', '77 min', '79 min',
       '157 m

In [28]:
mask = df_definitive['rating'].str.contains('min')
df_definitive.loc[mask, 'duration'] = df_definitive.loc[mask]['rating']
df_definitive.loc[mask, 'rating'] = 'no data'
mask2 = df_definitive['rating'].str.contains('eason')
df_definitive.loc[mask2, 'duration'] = df_definitive.loc[mask]['rating']
df_definitive.loc[mask2, 'rating'] = 'no data'
df_definitive.fillna('no data', inplace=True)

In [29]:
#Data normalization
df_definitive.columns = (['ID', 'Type', 'Tittle', 'Director', 'Cast', 'Country', 'Date_Added', 'Release_Year', 'Rating', 'Duration', 'Listed_in', 'Description', 'Platform'])

In [30]:
#Criteria unification
df_definitive['Rating'] = df_definitive['Rating'].replace({'no data'}, 'UNRATED')
df_definitive['Rating'] = df_definitive['Rating'].replace({'NOT_RATE'}, 'UNRATED')
df_definitive['Rating'] = df_definitive['Rating'].replace({'NOT RATED'}, 'UNRATED')
df_definitive['Rating'] = df_definitive['Rating'].replace({'NR'}, 'UNRATED')
df_definitive['Rating'] = df_definitive['Rating'].replace({'UR'}, 'UNRATED')

In [31]:
#Criteria modification
maskkk = df_definitive['Duration'].str.contains('min')
maskkk2 = df_definitive['Duration'].str.contains('Season')
df_definitive.loc[maskkk, 'Duration'] = df_definitive.loc[maskkk, 'Duration'].str[0:-4].astype(int)
df_definitive.loc[maskkk2, 'Duration'] = df_definitive[maskkk2]['Duration'].str[0:-7].astype(int)

In [32]:
def get_max_duration(year=int, platform=str, min_or_season=str):
    mask_movie = df_definitive['Type'] == 'Movie'
    mask_season = df_definitive['Type'] == 'TV Show'
    mask1 = df_definitive['Release_Year'] == year
    mask2 = df_definitive['Platform'] == platform
    mask3 = df_definitive['Duration'].str.contains('data') != True
    if min_or_season == "season":
        print(df_definitive.loc[mask1&mask2&mask_season, 'Duration'].sort_values().tail(1).to_dict())
    else:
        return df_definitive[mask1&mask2&mask3&mask_movie].sort_values('Duration')['Duration'].tail(1).to_dict()

get_max_duration(2018, "Hulu", "min")

{1949: 157}

In [33]:
def get_count_platform(platform):
    mask1 = df_definitive['Platform'] == platform
    mask_movie = df_definitive['Type'] == 'Movie'
    mask_season = df_definitive['Type'] == 'TV Show'
    if type(platform) != str:
        print('Please enter a string value: "Amazon_Prime", "Disney_Plus", "Hulu" or "Netflix"')
    else:
        return print(df_definitive[mask1&mask_movie].count()[0].tolist(), 'peliculas y', df_definitive[mask1&mask_season].count()[0], 'series')

get_count_platform("Amazon_Prime")

7814 peliculas y 1854 series


In [34]:
def get_listedin(genre):
    mas = df_definitive['Listed_in'].str.contains(genre, case=False)
    mas_amazonprime = df_definitive['Platform'] == 'Amazon_Prime'
    mas_disneyplus = df_definitive['Platform'] == 'Disney_Plus'
    mas_hulu = df_definitive['Platform'] == 'Hulu'
    mas_netflix = df_definitive['Platform'] == 'Netflix'
    if type(genre) != str:
        print('Please enter a string value')
    else:
        return f"Amazon Prime has {df_definitive[mas&mas_amazonprime].count()[0]}, Disney Plus {df_definitive[mas&mas_disneyplus].count()[0]}, Hulu {df_definitive[mas&mas_hulu].count()[0]} and Netflix {df_definitive[mas&mas_netflix].count()[0]}"
    
get_listedin('comedy')

'Amazon Prime has 2099, Disney Plus 526, Hulu 667 and Netflix 399'

In [55]:
from collections import Counter
import itertools

def get_actor(platform:str, year:int):
    draft = itertools.chain(*df_definitive['Cast'].str.split(','))
    hello = Counter(list(draft)).most_common()[1]
    mas_platform = df_definitive['Platform'] == platform
    mas_year = df_definitive['Release_Year'] == year
    draft2 = list(itertools.chain(*df_definitive[mas_platform&mas_year]['Cast'].str.split(',')))
    return f"The two actors who repeat themselves the most are {Counter(draft2).most_common()[0:2]}"

get_actor('Hulu', 2018)

"The two actors who repeat themselves the most are [('no data', 270)]"

In [92]:
def get_actor(platform:str, year:int):
    mas_platform = df_definitive['Platform'] == platform
    mas_year = df_definitive['Release_Year'] == year
    mas_nodata = df_definitive['Cast'].str.contains('no data') != True
    draft2 = list(itertools.chain(*df_definitive.loc[mas_platform&mas_year&mas_nodata, 'Cast'].str.split(',')))
    return f"The two actors who repeat themselves the most are {Counter(draft2).most_common()[0:2]}"

get_actor('Netflix', 2014)

"The two actors who repeat themselves the most are [(' Miyuki Sawashiro', 5), ('Mike Smith', 4)]"